# A Data Story Regarding Global Warming and Penguins

In [7]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from plotly.subplots import make_subplots

# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

import sys
!{sys.executable} -m pip install numpy

In [2]:
# Import datasets that will be used for the plots! Note that these datasets are already preprocessed (GitHub repo).
interpolated_data = pd.read_csv('interpolated_penguin_data.csv')
sealevels = pd.read_csv('sealevel_year.csv')
temp_df = pd.read_csv("Antarctica_temperatures.csv")
ice_df = pd.read_csv("iceberginfo_per_iceberg.csv")
new_temp_df = pd.read_csv("temp_per_year.csv")

# Perspective 1: Decreasing Penguin Habitat

## Correlation between global warming and Antarctica

In [3]:
temp_df_grouped = temp_df.groupby("Year").mean()

fig = px.scatter(temp_df_grouped, x="Year Code", y="Value", title='Temperature change in Antarctica (1961-2020)', 
                 labels={
                     "Year Code": "Year",
                     "Value": "Degree (C°)",
                 },
                 trendline='ols')
fig.update_traces(mode = 'lines')
fig.data[-1].line.color = 'red'
fig.show()

## Melting Iceberg A68A

In [4]:
ice_df = ice_df.groupby(["Iceberg", "Year"]).mean()
ice_df.to_csv("iceberg_per_year.csv")

ice_df = pd.read_csv("iceberg_per_year.csv")
spec_ice_df = ice_df[ice_df["Iceberg"] == "A68A"]

fig = px.bar(spec_ice_df, x="Year", y="Length (NM)", title='Average heights of icebergs in Antarctica', )
fig.show()

## Correlation between melting icebergs and rising sea levels

In [6]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=sealevels["year"], y=sealevels["GMSL_GIA"], name="Global Mean Sea Level (with GIA)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=sealevels["year"], y=new_temp_df["Value"], name="Temperature Change"),
    secondary_y=True,
)

fig.update_layout(
    title_text="Correlation between the temperature change in Antarctica and the average global sea levels (1993-2020)"
)

fig.update_xaxes(title_text="Year")

fig.update_yaxes(title_text="Sea level (mm)", secondary_y=False)
fig.update_yaxes(title_text="Degree (C°)", secondary_y=True)

fig.show()

# Perspective 2: Endangerment Of The Penguin Population

## Correlation between melting icebergs and penguin count

In [15]:
# Iliass

## Correlation between rising sea levels and number of penguins

In [16]:
total_penguin_2020 = interpolated_data[interpolated_data['year'] == 2020].\
    groupby('common_name')['penguin_count'].sum()

fig = go.Figure()
fig.add_trace(go.Pie(
    labels=total_penguin_2020.index,
    values=total_penguin_2020,
    textinfo='label+percent',
    textposition='outside',
    marker= {
      'colors': [
        '#11243B',
        '#EBF1F9',
        '#4682B4',
        '#B0C4DE',
        '#28548A',
        '#4D85CB',
      ]
    },
    hole=0.4,
    pull=[0.1, 0, 0, 0]
))

fig.update_layout(
    title="Distribution of Penguin Species",
    showlegend=False,
    height=600
)

fig.show()

### Figure Caption Here --

In [17]:
penguin_total = interpolated_data.groupby('year')['penguin_count'].sum()

fig = go.Figure([go.Bar(x=penguin_total.index, y=penguin_total, marker={'color': '#11243B'})])
fig.update_layout(title_text='Bar Plot of Total Penguin Count per Year', 
                  xaxis_tickangle=-45,
                  xaxis=go.layout.XAxis(
                    title="Year",
                    tickmode='linear',
                    tickangle = 45),
                  yaxis=go.layout.YAxis(
                    title="Total Penguins"),
                  height=550,
                  width= 1000
                 )
fig.show()

### Figure Caption Here

In [18]:
sealevels_range_2002_2020 = sealevels[(sealevels['year'] >= 2002) & (sealevels['year'] <= 2020)]
chinstrap_penguin = interpolated_data[interpolated_data['common_name'] == 'chinstrap penguin'].groupby('year')['penguin_count'].sum()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=chinstrap_penguin.index, y=chinstrap_penguin, name="Chinstrap Penguin Count", line=dict(color="#FF0000")),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=sealevels_range_2002_2020['year'], y=sealevels_range_2002_2020['GMSL_GIA'], name="Sealevels", line=dict(color="#11243B")),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Correlation between sealevels and the amount of penguins"
)

# Set x-axis title
fig.update_xaxes(title_text="<b>Year</b>", tickangle= 45)

# Set y-axes titles
fig.update_yaxes(title_text="<b>Penguin Count</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Sealevels</b>", secondary_y=True)

fig.update_layout(xaxis=dict(tickvals=sealevels_range_2002_2020['year']),
                autosize=False,
                width=990,
                height=600
            )

fig.show()

### Figure Caption Here

## Correlation between rising sea levels and number of Gentoo penguins

In [19]:
gentoo_penguin = interpolated_data[interpolated_data['common_name'] == 'gentoo penguin'].groupby('year')['penguin_count'].sum()
sealevels_range_2002_2020['GMSL_GIA']

fig = px.scatter_3d(gentoo_penguin, x=gentoo_penguin.index, y=gentoo_penguin, z=sealevels_range_2002_2020['GMSL_GIA'])

fig.update_layout(scene=dict(xaxis_title='Years', yaxis_title='Penguin Count', zaxis_title='TotalWeighted'),
                  title='Gentoo Penguin Count and Sealevels by Year',
                  height = 600)

fig.show()

## Correlation between rising sea levels and number of Emperor penguins

In [20]:
emperor_penguin = interpolated_data[interpolated_data['common_name'] == 'emperor penguin'].groupby('year')['penguin_count'].sum()
sealevels_range_2002_2020['GMSL_GIA']

fig = px.scatter_3d(emperor_penguin.index, x='year', y=emperor_penguin, z=sealevels_range_2002_2020['GMSL_GIA'])

fig.update_layout(scene=dict(xaxis_title='Years', yaxis_title='Penguin Count', zaxis_title='TotalWeighted'),
                  title='Emperor Penguin Count and Sealevels by Year',
                  height = 600)

fig.show()